# 📝 Exercise M3.01

The goal is to write an exhaustive search to find the best parameters
combination maximizing the model generalization performance.

Here we use a small subset of the Adult Census dataset to make the code
faster to execute. Once your code works on the small subset, try to
change `train_size` to a larger value (e.g. 0.8 for 80% instead of
20%).

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("../datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42)

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer(
    [('cat_preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))
])


Use the previously defined model (called `model`) and using two nested `for`
loops, make a search of the best combinations of the `learning_rate` and
`max_leaf_nodes` parameters. In this regard, you will need to train and test
the model by setting the parameters. The evaluation of the model should be
performed using `cross_val_score` on the training set. We will use the
following parameters search:
- `learning_rate` for the values 0.01, 0.1, 1 and 10. This parameter controls
  the ability of a new tree to correct the error of the previous sequence of
  trees
- `max_leaf_nodes` for the values 3, 10, 30. This parameter controls the
  depth of each tree.

In [3]:
model.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                     transformers=[('cat_preprocessor',
                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                   unknown_value=-1),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x7fb80ccf3e20>)])),
  ('classifier', HistGradientBoostingClassifier(random_state=42))],
 'verbose': False,
 'preprocessor': ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                   transformers=[('cat_preprocessor',
                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                 unknown_value=-1),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb80ccf3e20>)]),
 'classifier': HistGradientBoostingClassifier(random_st

In [8]:
from sklearn.model_selection import cross_val_score

learning_rate_list = [0.01, 0.1, 1, 10]
max_leaf_nodes_list = [3, 10, 30]

scores = []
for rate in learning_rate_list:
    for max_node in max_leaf_nodes_list:
        model.set_params(classifier__learning_rate=rate, classifier__max_leaf_nodes=max_node)
        score = cross_val_score(model, data_train, target_train, cv=2)
        # fit_params={'classifier__learning_rate':rate,'classifier__max_leaf_nodes':max_node})
        scores.append([rate, max_node, score.mean()])
        
scores
# 'classifier__learning_rate'
# 'classifier__max_leaf_nodes'

[[0.01, 3, 0.7892096642096642],
 [0.01, 10, 0.8134725634725635],
 [0.01, 30, 0.8417280917280917],
 [0.1, 3, 0.847051597051597],
 [0.1, 10, 0.8593366093366093],
 [0.1, 30, 0.8574938574938575],
 [1, 3, 0.8522727272727273],
 [1, 10, 0.8327190827190827],
 [1, 30, 0.8279074529074529],
 [10, 3, 0.2882882882882883],
 [10, 10, 0.4799344799344799],
 [10, 30, 0.6385135135135135]]

In [12]:
best = 0
best_row = []
for row in scores:
    if row[2] > best:
        best = row[2]
        best_row = row
best_row

[0.1, 10, 0.8593366093366093]


Now use the test set to score the model using the best parameters
that we found using cross-validation in the training set.

In [13]:
model.set_params(classifier__learning_rate=best_row[0], classifier__max_leaf_nodes=best_row[1])
final_score = cross_val_score(model, data_test, target_test, cv=2)
final_score

array([0.87219123, 0.86932487])

In [14]:
final_score.mean()

0.8707580488304243